In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelo lineal multivariante: evaluacion de modelos 

# Importando librerias:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms


# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/salarios.csv'

# Cargar los datos en un DataFrame
datos = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos.head())

In [ ]:
# Definir las variables independientes y dependientes
X = datos[['educacion', 'educacion_posgrado']]
y = datos['salario']

# Añadir una constante a las variables independientes
X = sm.add_constant(X)

# Ajustar el modelo de regresión lineal
model = sm.OLS(y, X).fit()

# Resumen del modelo
print(model.summary())

In [ ]:
# Ajustando el modelo de regresion utilizando una formula:
# resultados = smf.ols("np.log(salario) ~ np.log(educacion) + np.log(edad) + educacion_posgrado", data=datos).fit()
resultados = smf.ols("salario ~ educacion + educacion_posgrado", data=datos).fit()

# Resultados
print(resultados.summary())

In [ ]:
# Test de linealidad: Harvey-Collier
test = sms.linear_harvey_collier(resultados)
print(f'Estadígrafo HC: {test[0]}')
print(f'p-value HC ~ t(n-k-1): {test[1]}')

In [ ]:
# Test de normalidad de residuos: Jarque-Bera
test = sms.jarque_bera(resultados.resid)
print(f'Estadígrafo Jarque-Bera (JB): {test[0]}')
print(f'p-value JB ~ chi^2(2): {test[1]}')
print(f'Sesgo: {test[2]}')
print(f'Curtosis: {test[3]}')

In [ ]:
# Test Homocedasticidad: Breusch–Pagan
test = sms.het_breuschpagan(resultados.resid, resultados.model.exog)
print(f'Estadígrafo LMBP: {test[0]}')
print(f'p-value LMBP ~ chi^2(k-1): {test[1]}')
print(f'Estadígrafo FBP: {test[2]}')
print(f'p-value FBP ~ F(k,n-k-1): {test[3]}')

In [ ]:
# Test Homocedasticidad: Goldfeld-Quandt
test = sms.het_goldfeldquandt(resultados.resid, resultados.model.exog)
print(f'Estadígrafo FGQ: {test[0]}')
print(f'p-value FGQ ~ F(n1-k,n2-k): {test[1]}')

In [ ]:
# Test Homocedasticidad: White
from statsmodels.stats.diagnostic import het_white
white_test = het_white(resultados.resid, resultados.model.exog)
print(f'Estadístico LMW: {white_test[0]}')
print(f'p-value LMW ~ chi^2(k-1): {white_test[1]}')

In [ ]:
# Calcular errores estándar robustos usando la matriz de covarianza de tipo HC3
robust_cov = model.get_robustcov_results(cov_type='HC3')

# Mostrar los resultados con errores estándar robustos
print(robust_cov.summary())

In [ ]:
# Estimacion con minimos cuadrados ponderados (WLS):
import matplotlib.pyplot as plt

# Calcular los residuos del modelo OLS
residuos = model.resid

# Calcular los pesos inversamente proporcionales a los residuos al cuadrado (o a otra medida de varianza)
pesos = 1 / (residuos**2)

# Ajustar el modelo de regresión ponderada usando WLS
wls_model = sm.WLS(y, X, weights=pesos).fit()

# Mostrar el resumen del modelo OLS
print("Resumen del modelo OLS:")
print(model.summary())

# Mostrar el resumen del modelo WLS
print("\nResumen del modelo WLS:")
print(wls_model.summary())

# Graficar los residuos para observar la heterocedasticidad
plt.scatter(model.fittedvalues, residuos)
plt.xlabel('Valores Ajustados')
plt.ylabel('Residuos')
plt.title('Gráfico de Residuos (OLS)')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

# Graficar los residuos ponderados para observar la heterocedasticidad corregida
plt.scatter(wls_model.fittedvalues, wls_model.resid)
plt.xlabel('Valores Ajustados')
plt.ylabel('Residuos Ponderados')
plt.title('Gráfico de Residuos Ponderados (WLS)')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()
